In [15]:
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter
from datetime import datetime
from openpyxl.styles import Border, Side

# Tạo dữ liệu

In [3]:
wb = Workbook()
ws = wb.active
ws.title = "Bảng lương"
wb.save('Bang_luong.xlsx')

In [19]:
ws.merge_cells("A1:I1")
ws['A1'] = "Lương nhân viên"
ws['A1'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
ws["A1"].font = Font(size=13, bold=True, color="000000")
wb.save('Bang_luong.xlsx')

In [5]:
ws.append(['Mã số', 'Tên', 'Ngày bắt đầu đi làm', 'Số ca đạt doanh thu',
           'Số giờ làm', 'Lương mỗi giờ', 'Lương cơ bản', 'Thưởng doanh thu', 
           'Lương thực nhận'])
ws.append([1, 'Nguyễn Văn A', '28/4/2024', 5, 125])
ws.append([2, 'Nguyễn Thị B', '2/10/2024', 7, 130])
ws.append([3, 'Lê Văn C', '1/4/2025', 3, 72])
ws.append([4, 'Đinh Văn D', '8/1/2025', 5, 112])
ws.append([5, 'Nguyễn Thị E', '28/10/2024', 4, 103])
ws.append([6, 'Trần Thị E', '20/9/2024', 0, 64.7])
ws.append([7, 'Nguyễn Thị Y', '17/5/2024', 2, 79])
ws.append([8, 'Nguyễn Văn U', '2/8/2024', 3, 122.4])
ws.append([9, 'Võ Văn V', '10/2/2025', 7, 142])
wb.save('Bang_luong.xlsx')

In [ ]:
# Định dạng thành tiền VND
for row in ws.iter_rows(min_row=3, min_col=6, max_col=9):
    for cell in row:
        cell.number_format = '#,##0" VND"'  
wb.save("Bang_luong.xlsx")

In [7]:
# Định dạng lương theo cách tìm tên
# Tìm cột có tiêu đề trong hàng 2
header_row = 2
target_headers = ['Lương mỗi giờ', 'Lương cơ bản', 'Thưởng doanh thu', 'Lương thực nhận']

col_indexes = {}  # Từ điển lưu tên cột -> chỉ số

for col in range(1, ws.max_column + 1):
    cell_value = ws.cell(row=header_row, column=col).value
    if cell_value in target_headers:
        col_indexes[cell_value] = col

# Định dạng các cột theo tên đã tìm được
for header, col in col_indexes.items():
    col_letter = get_column_letter(col)
    for row in range(3, ws.max_row + 1):  # Bỏ qua tiêu đề
        ws[f"{col_letter}{row}"].number_format = '#,##0" VND"'

In [ ]:
# Định dạng ngày tháng
for row in ws.iter_rows(min_row=3, min_col=3, max_col=3):
    for cell in row:
        cell.number_format = 'DD/MM/YYYY'  
wb.save("Bang_luong.xlsx")

In [ ]:
# Hàm tính số tháng làm việc
def tinh_so_thang(cell):
    # Lấy giá trị ngày tháng từ ô
    bat_dau = cell.value
    bat_dau = datetime.strptime(bat_dau, "%d/%m/%Y") # Cần chuyển sang kiểu datetime
    # Kiểm tra nếu giá trị là kiểu ngày tháng (datetime)
    if isinstance(bat_dau, datetime):
        # Ngày hiện tại
        today = datetime.today()
        
        # Tính số tháng làm việc dựa trên sự chênh lệch ngày
        years_diff = today.year - bat_dau.year
        months_diff = today.month - bat_dau.month
        total_months = years_diff * 12 + months_diff
        
        # Nếu ngày trong tháng hiện tại chưa đến ngày bắt đầu làm việc, giảm tháng đi 1
        if today.day < bat_dau.day:
            total_months -= 1
        
        return total_months
    else:
        return 0  # Trả về 0 nếu không phải ngày hợp lệ

In [ ]:
# Thiết lập lương mỗi giờ
# Duyệt qua dữ liệu và tính số tháng làm việc
for i in range(3, ws.max_row + 1):  # Bắt đầu từ hàng 3
    start_date_cell = ws[f'C{i}']  # Cột C chứa ngày bắt đầu
    so_thang = tinh_so_thang(start_date_cell)
    # Ghi lương cơ bản (lương cơ bản ở cột F)
    if so_thang < 1:
        ws[f'F{i}'] = 18000
    elif so_thang > 6:
        ws[f'F{i}'] = 21000
    else:
        ws[f'F{i}'] = 20000
    
wb.save("Bang_luong.xlsx")

In [17]:
# Viền bảng
max_row = ws.max_row
max_col = ws.max_column

thin = Side(border_style="thin", color="000000")
border = Border(left=thin, right=thin, top=thin, bottom=thin)

# Áp dụng border cho từng ô trong vùng
for row in ws.iter_rows(min_row=1, max_row=max_row, min_col=0, max_col=max_col):
    for cell in row:
        cell.border = border

wb.save("Bang_luong.xlsx")

# Tính toán

In [ ]:
# Thiêt lập các cột cho việc tính toán
# Tìm cột có tiêu đề trong hàng 2
header_row = 2
target_headers = ['Số ca đạt doanh thu', 'Số giờ làm', 'Lương mỗi giờ', 
                  'Lương cơ bản', 'Thưởng doanh thu', 'Lương thực nhận']

col_indexes = {}  # Từ điển lưu tên cột -> chỉ số

for col in range(1, ws.max_column + 1):
    cell_value = ws.cell(row=header_row, column=col).value
    if cell_value in target_headers:
        col_indexes[cell_value] = col

# Lấy chỉ số cột
col_ca = col_indexes['Số ca đạt doanh thu']
col_gio = col_indexes['Số giờ làm']
col_luong_gio = col_indexes['Lương mỗi giờ']
col_luong_cb = col_indexes['Lương cơ bản']
col_thuong = col_indexes['Thưởng doanh thu']
col_luong = col_indexes['Lương thực nhận']

In [12]:
#Lương cơ bản
for row in range(3, ws.max_row + 1):  # Duyệt từ dòng chứa dữ liệu đầu tiên
    gio_lam = ws.cell(row=row, column=col_gio).value
    luong_moi_gio = ws.cell(row=row, column=col_luong_gio).value

    if gio_lam is not None and luong_moi_gio is not None:
        luong_cb = gio_lam * luong_moi_gio
        ws.cell(row=row, column=col_luong_cb).value = luong_cb

wb.save("Bang_luong.xlsx")

In [ ]:
# Tính thưởng doanh thu
for row in range(3, ws.max_row + 1):  
    ca = ws.cell(row=row, column=col_ca).value

    if ca is not None:
        thuong = ca * 50000
        ws.cell(row=row, column=col_thuong).value = thuong 

wb.save("Bang_luong.xlsx")

In [ ]:
# Tính lương thực nhận 
for row in range(3, ws.max_row + 1):  
    luong_cb = ws.cell(row=row, column=col_luong_cb).value
    thuong = ws.cell(row=row, column=col_thuong).value

    if luong_cb is not None and thuong is not None:
        luong = luong_cb + thuong - luong_cb*0.1
        ws.cell(row=row, column=col_luong).value = luong

wb.save("Bang_luong.xlsx")